# 👗 Projet Fashion-MNIST : Transfer Learning

## 🎯 Objectifs

Dans ce projet complet, nous appliquons tout ce que nous avons appris sur un **nouveau dataset** :

- 👕 **Fashion-MNIST** : Classification de vêtements (au lieu de chiffres)
- 🔄 **Transfer Learning** : Réutiliser nos connaissances
- 🚀 **Projet End-to-End** : De l'exploration au déploiement
- 📊 **Comparaison** : MNIST vs Fashion-MNIST
- 🎓 **Apprentissage** : Généralisation des concepts

---

## 🤔 Pourquoi Fashion-MNIST ?

### Problème avec MNIST

❌ **Trop facile** : ~99% d'accuracy facile à atteindre  
❌ **Trop simple** : Images très propres, peu de variabilité  
❌ **Sur-utilisé** : Risque de suroptimisation dans la communauté

### Fashion-MNIST : Le "MNIST difficile"

✅ **Plus challengeant** : ~90-93% d'accuracy (vs 98-99% pour MNIST)  
✅ **Plus réaliste** : Images de vêtements avec plus de variabilité  
✅ **Même format** : 28×28 grayscale, drop-in replacement  
✅ **Moderne** : Publié en 2017 par Zalando Research

### Les 10 Classes

| ID | Label FR | Label EN |
|----|----------|----------|
| 0 | T-shirt/Top | T-shirt/top |
| 1 | Pantalon | Trouser |
| 2 | Pull | Pullover |
| 3 | Robe | Dress |
| 4 | Manteau | Coat |
| 5 | Sandale | Sandal |
| 6 | Chemise | Shirt |
| 7 | Basket | Sneaker |
| 8 | Sac | Bag |
| 9 | Bottine | Ankle boot |

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import time
import gzip
import urllib.request

# Ajouter le dossier parent au path
sys.path.append('../')

from src.network import NeuralNetwork
from src import visualize
from src.metrics import (
    accuracy, confusion_matrix, precision, recall, f1_score,
    classification_report, print_classification_report
)

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Imports réussis !")

## 1️⃣ Téléchargement et Chargement de Fashion-MNIST

Fashion-MNIST utilise le **même format** que MNIST, ce qui facilite le transfer learning.

In [ ]:
def download_fashion_mnist(data_dir='../data/fashion_mnist'):
    """
    Télécharge le dataset Fashion-MNIST
    """
    data_dir = Path(data_dir)
    data_dir.mkdir(parents=True, exist_ok=True)
    
    base_url = 'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/'
    files = [
        'train-images-idx3-ubyte.gz',
        'train-labels-idx1-ubyte.gz',
        't10k-images-idx3-ubyte.gz',
        't10k-labels-idx1-ubyte.gz'
    ]
    
    for file in files:
        filepath = data_dir / file
        if not filepath.exists():
            print(f"Téléchargement de {file}...")
            urllib.request.urlretrieve(base_url + file, filepath)
            print(f"  ✓ {file} téléchargé")
        else:
            print(f"  ✓ {file} déjà présent")
    
    return data_dir

def load_fashion_mnist_images(filepath):
    """Charge les images Fashion-MNIST"""
    with gzip.open(filepath, 'rb') as f:
        # Lire l'en-tête (16 bytes)
        magic = int.from_bytes(f.read(4), 'big')
        n_images = int.from_bytes(f.read(4), 'big')
        n_rows = int.from_bytes(f.read(4), 'big')
        n_cols = int.from_bytes(f.read(4), 'big')
        
        # Lire les données
        data = np.frombuffer(f.read(), dtype=np.uint8)
        data = data.reshape(n_images, n_rows * n_cols)
        
        # Normaliser [0, 255] → [0, 1]
        data = data.astype(np.float32) / 255.0
        
    return data

def load_fashion_mnist_labels(filepath):
    """Charge les labels Fashion-MNIST"""
    with gzip.open(filepath, 'rb') as f:
        # Lire l'en-tête (8 bytes)
        magic = int.from_bytes(f.read(4), 'big')
        n_labels = int.from_bytes(f.read(4), 'big')
        
        # Lire les labels
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        
    return labels

def load_fashion_mnist_data(data_dir='../data/fashion_mnist', val_split=0.1):
    """
    Charge Fashion-MNIST et le découpe en train/val/test
    """
    data_dir = Path(data_dir)
    
    # Charger train
    X_train_full = load_fashion_mnist_images(data_dir / 'train-images-idx3-ubyte.gz')
    y_train_full = load_fashion_mnist_labels(data_dir / 'train-labels-idx1-ubyte.gz')
    
    # Charger test
    X_test = load_fashion_mnist_images(data_dir / 't10k-images-idx3-ubyte.gz')
    y_test = load_fashion_mnist_labels(data_dir / 't10k-labels-idx1-ubyte.gz')
    
    # Séparer train/val
    n_val = int(len(X_train_full) * val_split)
    
    indices = np.random.permutation(len(X_train_full))
    val_indices = indices[:n_val]
    train_indices = indices[n_val:]
    
    X_train = X_train_full[train_indices]
    y_train = y_train_full[train_indices]
    X_val = X_train_full[val_indices]
    y_val = y_train_full[val_indices]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

print("✓ Fonctions de chargement définies")

In [ ]:
# Télécharger et charger Fashion-MNIST
print("📥 Téléchargement de Fashion-MNIST...\n")
data_dir = download_fashion_mnist()

print("\n📂 Chargement des données...\n")
X_train, y_train, X_val, y_val, X_test, y_test = load_fashion_mnist_data()

print(f"\n✓ Données chargées:")
print(f"  Train: {X_train.shape[0]:,} exemples")
print(f"  Val:   {X_val.shape[0]:,} exemples")
print(f"  Test:  {X_test.shape[0]:,} exemples")
print(f"  Shape: {X_train.shape[1]} features (28×28)")

---

## 2️⃣ Exploration du Dataset

In [ ]:
# Noms des classes
class_names = [
    'T-shirt/Top',
    'Pantalon',
    'Pull',
    'Robe',
    'Manteau',
    'Sandale',
    'Chemise',
    'Basket',
    'Sac',
    'Bottine'
]

In [ ]:
# Visualiser des échantillons
n_samples = 25
sample_indices = np.random.choice(len(X_train), n_samples, replace=False)

fig, axes = plt.subplots(5, 5, figsize=(12, 12))
axes = axes.flatten()

for i, idx in enumerate(sample_indices):
    ax = axes[i]
    image = X_train[idx].reshape(28, 28)
    label = y_train[idx]
    
    ax.imshow(image, cmap='gray')
    ax.set_title(f'{class_names[label]} ({label})', fontsize=9)
    ax.axis('off')

plt.suptitle('Échantillons de Fashion-MNIST', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../models/fashion_mnist_samples.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Distribution des classes
visualize.plot_class_distribution(
    y_train, 
    class_names=class_names,
    figsize=(12, 6),
    save_path='../models/fashion_mnist_distribution.png'
)

### 🔍 Analyse de Difficulté

Certaines classes sont **visuellement similaires** :

- **T-shirt vs Chemise vs Pull** → Tous sont des vêtements du haut
- **Basket vs Bottine vs Sandale** → Tous sont des chaussures
- **Manteau vs Pull** → Formes similaires

→ Fashion-MNIST est **plus difficile** que MNIST !

In [ ]:
# Visualiser un échantillon de chaque classe
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for class_id in range(10):
    # Trouver un exemple de cette classe
    idx = np.where(y_train == class_id)[0][0]
    image = X_train[idx].reshape(28, 28)
    
    ax = axes[class_id]
    ax.imshow(image, cmap='gray')
    ax.set_title(f'{class_id}: {class_names[class_id]}', fontsize=11, fontweight='bold')
    ax.axis('off')

plt.suptitle('Les 10 Classes de Fashion-MNIST', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../models/fashion_mnist_classes.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 3️⃣ Baseline : Modèle Simple

Commençons avec une architecture simple pour établir une **baseline**.

In [ ]:
print("🚀 Entraînement du modèle baseline...\n")

baseline_model = NeuralNetwork(
    layer_dims=[784, 128, 64, 10],
    learning_rate=0.01,
    optimizer='adam'
)

baseline_model.train(
    X_train, y_train, X_val, y_val,
    epochs=15,
    batch_size=128,
    verbose=True
)

# Évaluer
baseline_acc = baseline_model.accuracy(X_test, y_test)
print(f"\n🎯 Accuracy Baseline: {baseline_acc:.4f}")

In [ ]:
# Visualiser l'apprentissage
visualize.plot_training_history(
    baseline_model.history,
    figsize=(14, 5),
    save_path='../models/fashion_baseline_training.png'
)

---

## 4️⃣ Amélioration : Modèle Optimisé

Maintenant, appliquons ce que nous avons appris pour améliorer les performances :

1. **Architecture plus profonde**
2. **Learning rate adapté**
3. **Plus d'époques**
4. **Optimizer Adam**

In [ ]:
print("🚀 Entraînement du modèle optimisé...\n")

optimized_model = NeuralNetwork(
    layer_dims=[784, 256, 128, 64, 10],
    learning_rate=0.01,
    optimizer='adam'
)

optimized_model.train(
    X_train, y_train, X_val, y_val,
    epochs=25,
    batch_size=128,
    verbose=True
)

# Évaluer
optimized_acc = optimized_model.accuracy(X_test, y_test)
print(f"\n🎯 Accuracy Optimisé: {optimized_acc:.4f}")
print(f"Gain vs Baseline: {optimized_acc - baseline_acc:.4f} (+{(optimized_acc/baseline_acc - 1)*100:.2f}%)")

In [ ]:
# Comparer les courbes d'apprentissage
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
epochs_baseline = range(1, len(baseline_model.history['loss']) + 1)
epochs_optimized = range(1, len(optimized_model.history['loss']) + 1)

ax1.plot(epochs_baseline, baseline_model.history['loss'], 'b-o', 
         linewidth=2, markersize=4, label='Baseline')
ax1.plot(epochs_optimized, optimized_model.history['loss'], 'r-o', 
         linewidth=2, markersize=4, label='Optimisé')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Comparaison Loss', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(epochs_baseline, baseline_model.history['val_acc'], 'b-o', 
         linewidth=2, markersize=4, label='Baseline')
ax2.plot(epochs_optimized, optimized_model.history['val_acc'], 'r-o', 
         linewidth=2, markersize=4, label='Optimisé')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy', fontsize=12)
ax2.set_title('Comparaison Accuracy', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../models/fashion_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 5️⃣ Analyse Détaillée des Performances

In [ ]:
# Prédictions
y_pred = optimized_model.predict(X_test)
y_probs, _ = optimized_model.forward(X_test)

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred, num_classes=10)

visualize.plot_confusion_matrix(
    cm,
    class_names=class_names,
    figsize=(14, 6),
    save_path='../models/fashion_confusion_matrix.png'
)

In [ ]:
# Rapport de classification complet
print_classification_report(y_test, y_pred, class_names=class_names)

### 🔍 Analyse des Confusions

Identifions les paires les plus confondues.

In [ ]:
# Top confusions
print("\n🔍 Top 10 Paires les Plus Confondues:\n")

confusions = []
for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 0:
            confusions.append((i, j, cm[i, j]))

confusions.sort(key=lambda x: x[2], reverse=True)

print(f"{'Vrai':<15} {'Prédit':<15} {'Erreurs':<10} {'% du vrai'}")
print("-" * 55)

for true, pred, count in confusions[:10]:
    total_true = cm[true, :].sum()
    percentage = (count / total_true) * 100
    print(f"{class_names[true]:<15} {class_names[pred]:<15} {count:<10} {percentage:.2f}%")

print("\n💡 Observations:")
print("- Les vêtements du haut (T-shirt, Chemise, Pull) sont souvent confondus")
print("- Les chaussures (Basket, Bottine, Sandale) aussi")
print("- C'est logique : ils sont visuellement similaires !")

### 🎨 Visualisation des Prédictions

In [ ]:
# Exemples de prédictions
visualize.plot_sample_predictions(
    X_test,
    y_test,
    y_pred,
    y_probs,
    n_samples=25,
    figsize=(14, 14),
    save_path='../models/fashion_predictions.png'
)

In [ ]:
# Exemples mal classifiés
visualize.plot_misclassified_examples(
    X_test,
    y_test,
    y_pred,
    y_probs,
    n_samples=20,
    figsize=(14, 12),
    save_path='../models/fashion_errors.png'
)

---

## 6️⃣ Comparaison MNIST vs Fashion-MNIST

### 📊 Difficulté Relative

In [ ]:
# Résumé des performances
print("\n" + "="*70)
print("📊 COMPARAISON MNIST vs FASHION-MNIST")
print("="*70 + "\n")

# Charger ou définir les résultats MNIST (à partir des notebooks précédents)
mnist_baseline = 0.96  # Architecture similaire sur MNIST
mnist_optimized = 0.98  # Modèle optimisé sur MNIST

comparison = {
    'Dataset': ['MNIST', 'MNIST', 'Fashion-MNIST', 'Fashion-MNIST'],
    'Modèle': ['Baseline', 'Optimisé', 'Baseline', 'Optimisé'],
    'Accuracy': [mnist_baseline, mnist_optimized, baseline_acc, optimized_acc]
}

print(f"{'Dataset':<20} {'Modèle':<15} {'Accuracy'}")
print("-" * 50)
for dataset, model, acc in zip(comparison['Dataset'], comparison['Modèle'], comparison['Accuracy']):
    print(f"{dataset:<20} {model:<15} {acc:.4f}")

print("\n💡 Observations:")
print(f"- Fashion-MNIST est ~{(mnist_optimized - optimized_acc)*100:.1f}% plus difficile que MNIST")
print("- Les mêmes techniques d'optimisation fonctionnent sur les deux")
print("- Fashion-MNIST est un meilleur benchmark pour tester les algorithmes")

In [ ]:
# Visualisation comparative
fig, ax = plt.subplots(figsize=(10, 6))

datasets = ['MNIST\nBaseline', 'MNIST\nOptimisé', 'Fashion-MNIST\nBaseline', 'Fashion-MNIST\nOptimisé']
accuracies = comparison['Accuracy']
colors = ['lightblue', 'blue', 'lightcoral', 'red']

bars = ax.bar(datasets, accuracies, color=colors, edgecolor='black', linewidth=2)

for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.4f}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Comparaison MNIST vs Fashion-MNIST', fontsize=14, fontweight='bold')
ax.set_ylim([0.80, 1.0])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../models/mnist_vs_fashion_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 7️⃣ Sauvegarde du Modèle

Sauvegardons notre meilleur modèle pour une utilisation future.

In [ ]:
# Sauvegarder le modèle optimisé
model_path = Path('../models/fashion_mnist_best.pkl')
optimized_model.save(model_path)

print(f"\n✓ Modèle sauvegardé: {model_path}")
print(f"  Architecture: {' → '.join(map(str, optimized_model.layer_dims))}")
print(f"  Test Accuracy: {optimized_acc:.4f}")
print(f"  Optimizer: {optimized_model.optimizer_name}")

In [ ]:
# Test de chargement
print("\n🔄 Test de chargement du modèle...")
loaded_model = NeuralNetwork.load(model_path)

# Vérifier que ça fonctionne
loaded_acc = loaded_model.accuracy(X_test, y_test)
print(f"  Accuracy après chargement: {loaded_acc:.4f}")
print(f"  ✓ Modèle chargé correctement !" if abs(loaded_acc - optimized_acc) < 1e-6 else "  ❌ Erreur !")

---

## 🎓 Conclusions et Apprentissages

### ✅ Ce que nous avons accompli

1. ✅ **Chargé un nouveau dataset** (Fashion-MNIST)
2. ✅ **Appliqué nos connaissances** sur un problème différent
3. ✅ **Comparé les performances** avec MNIST
4. ✅ **Analysé les erreurs** et les confusions
5. ✅ **Optimisé le modèle** pour de meilleures performances
6. ✅ **Sauvegardé le modèle** pour réutilisation

### 🎯 Résultats

| Métrique | Valeur |
|----------|--------|
| **Baseline Accuracy** | ~85-87% |
| **Optimisé Accuracy** | ~88-91% |
| **Amélioration** | +3-5% |
| **Classes les plus difficiles** | T-shirt, Chemise, Pull |
| **Classes les plus faciles** | Pantalon, Sac |

### 💡 Leçons Apprises

1. **Transfer Learning fonctionne** : Les mêmes techniques marchent sur différents datasets
2. **Fashion-MNIST est plus difficile** : ~6-8% moins d'accuracy que MNIST
3. **Similarité visuelle = Confusions** : Classes similaires sont confondues
4. **Architecture profonde aide** : Plus de couches = meilleures performances
5. **Adam est robuste** : Fonctionne bien sur différents problèmes

### 🚀 Pour Aller Plus Loin

**Améliorations possibles** :

1. **Data Augmentation** : Rotation, translation, zoom
2. **CNN** : Réseaux convolutifs (mieux pour les images)
3. **Ensemble Methods** : Combiner plusieurs modèles
4. **Hyperparameter Tuning** : Optimiser LR, batch size, architecture
5. **Régularisation** : Dropout, L2 pour réduire le surapprentissage

**Target** : Avec CNN et optimisations, on peut atteindre **~92-93% accuracy** sur Fashion-MNIST.

---

## 🎯 Exercices

1. **Implémenter data augmentation** et mesurer l'amélioration
2. **Tester différentes architectures** et comparer
3. **Analyser les features apprises** (visualisation des poids)
4. **Créer un ensemble** de 3-5 modèles
5. **Implémenter un CNN** simple et comparer avec MLP
6. **Mesurer l'impact du learning rate** sur Fashion-MNIST

---

**Félicitations ! 🎉**

Vous avez maintenant l'expérience de travailler sur un **projet complet** de classification d'images !

Vous savez comment :
- Charger et explorer un nouveau dataset
- Adapter vos modèles à de nouveaux problèmes
- Analyser et améliorer les performances
- Sauvegarder et déployer vos modèles

Ces compétences sont **directement transférables** à d'autres problèmes de machine learning ! 🚀